In [11]:
import yfinance as yf
from datetime import datetime
import pandas as pd
import psycopg2
import configparser
import datetime
import nbimporter
from General import *
from Daily_etl import *
from SQL import *
from Functions import *

ImportError: cannot import name 'HTMLParserTreeBuilder' from 'bs4.builder' (C:\Users\agd986\AppData\Local\anaconda3\Lib\site-packages\bs4\builder\__init__.py)

In [5]:
#This is a sample test and will be used for testing only. 
#It utilises yfinance library. It takes "Ticker" symbol then returns the ticker information. 
#Fetch data for a single stock symbol
#This is used to get financial information on stock market from yahoo finance using ticker as test 

stock = yf.Ticker("0DK9.L")
hist = stock.history(period="6y")  # Fetches one month of historical data

print(hist)
#If you want to see the result just uncomment the print statement.
#This was a test that was run to ensure that the code is able to work on one ticker


NameError: name 'yf' is not defined

In [2]:
#connecting to the database


def connect_to_database_AWS_postgreSQL():
    try:
        # Read database credentials from the configuration file
        config = configparser.ConfigParser()
        config.read('configuration.ini')

        host = config['database']['host']
        port = int(config['database']['port'])
        user = config['database']['user']
        password = config['database']['password']
        database = config['database']['database']

       # Establish a connection to the database using the credentials from the config file
        connection = psycopg2.connect(host=host, port=port, user=user, password=password, database=database)

        return connection

    except (Exception, psycopg2.Error) as error:
        print(f"Error: {error}")

if __name__ == "__main__":
    # Establish a connection to the database
    connection = connect_to_database_AWS_postgreSQL()

    if connection:
        print("Connected to the database successfully.")

        connection.close()
    else:
        print("Failed to connect to the database.")


Connected to the database successfully.


In [3]:

# Function to execute SQL queries
#SQL REFERENCE A


def execute_sql_query_AWS_postgreSQL(query, params=None):
    
    # Establish a connection to the database
    connection = connect_to_database_AWS_postgreSQL()

    try:
        if connection:
            cursor = connection.cursor()
            
            # Execute the SQL query
            if params:
                cursor.execute(query, params)
            else:
                cursor.execute(query)

            # If it's a SELECT query, fetch the results
            if query.strip().lower().startswith("select"):
                results = cursor.fetchall()
                cursor.close()
                return results

            # For other types of queries, commit the changes
            else:
                connection.commit()
               # print("Query executed successfully.")

    except (Exception, psycopg2.Error) as error:
        print(f"Error: {error}")
        if connection:
            connection.rollback()

    finally:
        if connection:
            connection.close()

In [16]:
TEST= """SELECT symbol, COUNT(DISTINCT date) AS date_count
FROM s_historical_stock_data
GROUP BY symbol;
"""

In [18]:
TEST = """SELECT * FROM s_historical_stock_data WHERE symbol = '0J0H.L' order by date DESC"""

In [292]:
TEST = """SELECT count (symbol) FROM s_historical_stock_data;"""

In [14]:
TEST = """SELECT distinct( symbol) FROM s_historical_stock_data WHERE symbol ilike 'E%'"""

In [36]:
TEST = """SELECT * from d_customer"""

In [37]:

# Function to execute SQL queries
#SQL REFERENCE A


execute_sql_query_AWS_postgreSQL(TEST)

[('1725 3rd Street',
  'San Francisco',
  'CA',
  '94158',
  'United States',
  '415 612 8582',
  'https://www.uber.com',
  None,
  None,
  None,
  '0A1U.L',
  'Uber Technologies, Inc.',
  30400,
  Decimal('0.05'),
  Decimal('1.27'),
  Decimal('0.15'),
  None,
  Decimal('51.61'),
  Decimal('22.17'),
  'USD',
  'LSE',
  None),
 (None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 (None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 ('3655 N.W. 87th Avenue',
  'Miami',
  'FL',
  '33178-2428',
  'United States',
  '305 599 2600',
  'https://www.carnivalcorp.com',
  None,
  None,
  None,
  '0EV1.L',
  'Carnival Corporation & plc',
  106000,
  Decimal('0.02'),
  None,
  Decimal('0.22'),
  Decimal('108358544.00'),
  Decimal('19.73'),
  Decimal('8.5

In [21]:
#*******
#Creatig function to fetch Information on copanies that are listed based on symbol.
#This is slightly different because it populates information on companies.


#Reference cell B
def fetch_info_data_AWS_postgreSQL(symbol):
    try:
        stock = yf.Ticker(symbol)
        info = stock.info

        selected_columns = [
            "address1", "city", "state", "zip", "country", "phone", "website",
            "industry", "sector", "sectorKey", "symbol", "longName",
            "fullTimeEmployees", "profitMargins", "earningsGrowth", "revenueGrowth",
            "marketCap", "fiftyTwoWeekHigh", "fiftyTwoWeekLow", "currency", "exchange"
        ]

        info_data = {column: info.get(column, "N/A") for column in selected_columns}
        return pd.DataFrame([info_data])

    except Exception as e:
        print(f"Error fetching data for {symbol}: {str(e)}")
        return None

In [73]:
#***************
# The below function inserts company data into source schema inside table s_customer table. This table was created directly inside postgreSQL
# This will be commented out to avoid running by error and rewriting the entire table or causing duplication.
#SQL REFERENCE B
#Reference cell A
#***************



def insert_customer_data_AWS_postgreSQL(row):
    query = """
    INSERT INTO s_customer 
    (Address1, City, State, Zip, Country, Phone, Website, Industry, Sector, SectorKey, Symbol, 
    Customer_Name, Full_Time_Employees, Profit_Margins, Earnings_Growth, Revenue_Growth, Market_Cap, 
    Fifty_Two_Week_High, Fifty_Two_Week_Low, Currency, Exchange) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    params = (
        row['address1'] if row['address1'] != 'N/A' else None,
        row['city'] if row['city'] != 'N/A' else None,
        row['state'] if row['state'] != 'N/A' else None,
        row['zip'] if row['zip'] != 'N/A' else None,
        row['country'] if row['country'] != 'N/A' else None,
        row['phone'] if row['phone'] != 'N/A' else None,
        row['website'] if row['website'] != 'N/A' else None,
        row['industry'] if row['industry'] != 'N/A' else None,
        row['sector'] if row['sector'] != 'N/A' else None,
        row['sectorKey'] if row['sectorKey'] != 'N/A' else None,
        row['symbol'] if row['symbol'] != 'N/A' else None,
        row['longName'] if row['longName'] != 'N/A' else None,
        int(row['fullTimeEmployees']) if row['fullTimeEmployees'] != 'N/A' else None,
        float(row['profitMargins']) if row['profitMargins'] != 'N/A' else None,
        float(row['earningsGrowth']) if row['earningsGrowth'] != 'N/A' else None,
        float(row['revenueGrowth']) if row['revenueGrowth'] != 'N/A' else None,
        float(row['marketCap']) if row['marketCap'] != 'N/A' else None,
        float(row['fiftyTwoWeekHigh']) if 'fiftyTwoWeekHigh' in row and row['fiftyTwoWeekHigh'] != 'N/A' else None,
        float(row['fiftyTwoWeekLow']) if 'fiftyTwoWeekLow' in row and row['fiftyTwoWeekLow'] != 'N/A' else None,
        row['currency'] if row['currency'] != 'N/A' else None,
        row['exchange'] if row['exchange'] != 'N/A' else None
    )
    execute_sql_query_AWS_postgreSQL(query, params)
    

In [13]:
#*****************
#This code uses both function above and loops through symbols in bathc of 100 and fetches their information 
#Then once information is fetched it is then inserted into the the respective table
#This code will be commented out. To avoid running by error and rewriting the entire table or causing duplication.
#SQL REFERENCE B
#Reference cell B
#*****************

'''
with open("lse.txt", "r", encoding='utf-8') as file:
    next(file)  # Skip the header line
    symbols = [line.split('\t')[0].strip() for line in file]

batch_size = 100
error_symbols = []

for i in range(0, len(symbols), batch_size):
    batch_symbols = symbols[i:i + batch_size]
    for symbol in batch_symbols:
        info_df = fetch_info_data_AWS_postgreSQL(symbol)
        if info_df is not None and not info_df.empty:
            for _, row in info_df.iterrows():
                insert_customer_data_AWS_postgreSQL(row)
        else:
            error_symbols.append(symbol)  # Append symbol to error list if data fetch fails

    print(f"Processed batch {i // batch_size + 1}/{len(symbols) // batch_size + 1}")


print("All data processed and inserted into source.s_customer")
if error_symbols:
    with open('s_customer_error_symbols.csv', 'w') as f:
        for symbol in error_symbols:
            f.write(f"{symbol}\n")
    print(f"Symbols with errors written to 's_customer_error_symbols.csv'")
'''


'\nwith open("lse.txt", "r", encoding=\'utf-8\') as file:\n    next(file)  # Skip the header line\n    symbols = [line.split(\'\t\')[0].strip() for line in file]\n\nbatch_size = 100\nerror_symbols = []\n\nfor i in range(0, len(symbols), batch_size):\n    batch_symbols = symbols[i:i + batch_size]\n    for symbol in batch_symbols:\n        info_df = fetch_info_data_AWS_postgreSQL(symbol)\n        if info_df is not None and not info_df.empty:\n            for _, row in info_df.iterrows():\n                insert_customer_data_AWS_postgreSQL(row)\n        else:\n            error_symbols.append(symbol)  # Append symbol to error list if data fetch fails\n\n    print(f"Processed batch {i // batch_size + 1}/{len(symbols) // batch_size + 1}")\n\n\nprint("All data processed and inserted into source.s_customer")\nif error_symbols:\n    with open(\'s_customer_error_symbols.csv\', \'w\') as f:\n        for symbol in error_symbols:\n            f.write(f"{symbol}\n")\n    print(f"Symbols with error